In [37]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize

prices = pd.read_excel('C:/Users/tjgus/Desktop/black_literman_data.xlsx')
prices = prices.set_index('month')

returns = prices.pct_change().dropna().reset_index(drop=True)
returns

,INTC,AEP,AMZN,MRK,XOM,S&P 500
0,-0.184704,0.066163,-0.006342,0.036376,0.063787,-0.020766
1,0.064887,0.051123,0.014184,-0.055906,0.061325,0.036739
2,-0.059192,-0.006466,0.167133,-0.056100,-0.083462,-0.061418
3,-0.033679,-0.054612,0.092271,0.050708,-0.000280,-0.009081
4,-0.338568,-0.063454,-0.108612,-0.107520,0.024944,-0.072455
...,...,...,...,...,...,...
61,-0.036943,0.085746,0.016607,0.008976,0.052683,0.009980
62,0.124015,0.030184,0.541342,0.164690,0.052043,0.043291
63,0.036933,-0.044150,0.127344,0.019585,0.052252,0.032549
64,0.070334,-0.054377,-0.010558,-0.043411,0.008537,-0.017816


In [20]:
returns_without_market = returns.drop(columns=['S&P 500'])

correlation_matrix = returns_without_market.corr()

covariance_matrix = returns_without_market.cov()
covariance_matrix

,INTC,AEP,AMZN,MRK,XOM
INTC,0.011346,0.000657,0.007338,0.001618,0.000708
AEP,0.000657,0.004273,0.001172,0.001147,0.001064
AMZN,0.007338,0.001172,0.020154,0.001519,-0.000016
MRK,0.001618,0.001147,0.001519,0.006086,0.001093
XOM,0.000708,0.001064,-0.000016,0.001093,0.002995


In [27]:
# risk-free data
rf_annual = 0.05
rf_monthly = rf_annual / 12 

mkt_rp_annual = 0.08
mkt_rp_monthly = mkt_rp_annual / 12  

# histirical data
historical_returns = returns.mean()
excess_returns_hist = historical_returns[:-1] - rf_monthly

# CAPM = risk free rate + market risk premium * beta
beta = pd.Series(dtype='float64')
x = returns['S&P 500'].values.reshape(-1, 1)

for col in returns.columns:
    y  = returns[col]
    model = LinearRegression()
    model.fit(x, y)
    beta_ = model.coef_[0]
    beta[col] = beta_

capm = rf_monthly + mkt_rp_monthly * beta[:-1]  # except market 
excess_returns_capm = capm - rf_monthly

excess_returns = pd.DataFrame()
excess_returns['historical'] = excess_returns_hist
excess_returns['capm'] = excess_returns_capm

excess_returns

,historical,capm
INTC,-0.003509,0.013544
AEP,0.002649,0.006426
AMZN,0.032674,0.011094
MRK,0.000212,0.005660
XOM,0.011129,0.005197


In [32]:
# optimal solution - excess returns / covariance matrix

inverse = np.linalg.inv(covariance_matrix.values)

z_hist = np.dot(inverse, excess_returns['historical'])
z_capm = np.dot(inverse, excess_returns['capm'])

z = pd.DataFrame()
z['historical'] = z_hist
z['capm'] = z_capm
z

,historical,capm
0,-2.054930,0.948911
1,-0.721531,0.985618
2,2.471716,0.128227
3,-0.751990,0.268956
4,4.746039,1.063362


In [38]:
# weight (unrestricted)

weight_u = pd.DataFrame()
weight_u['historical'] = z['historical'] / z['historical'].sum()
weight_u['capm'] = z['capm'] / z['capm'].sum()
weight_u

,historical,capm
0,-0.556997,0.279497
1,-0.195574,0.290308
2,0.669968,0.037769
3,-0.203830,0.079219
4,1.286432,0.313207


In [88]:
# weight (no short selling, sharpe ratio maximization)

def objective(weights, returns, cov_matrix):
    portfolio_return = np.sum(returns * weights)
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = -portfolio_return / portfolio_std  # 음수로 설정하여 최대화로 변환
    return sharpe_ratio

# 제약 조건: 비중의 합은 1
def constraint1(weights):
    return np.sum(weights) - 1

# 초기 추정치 (균등 비중)
initial_guess = np.array([1/len(excess_returns)] * len(excess_returns))

# 최적화 함수 호출
bounds = [(0, 1) for _ in range(len(excess_returns_hist))]  # 비중은 0과 1 사이여야 함
constraints = [{'type': 'eq', 'fun': constraint1}]

result_hist = minimize(objective, initial_guess, args=(excess_returns['historical'].values, covariance_matrix.values), method='SLSQP', bounds=bounds, constraints=constraints)
result_capm = minimize(objective, initial_guess, args=(excess_returns['capm'].values, covariance_matrix.values), method='SLSQP', bounds=bounds, constraints=constraints)


# 결과 출력
print("최적 비중:", result_hist.x)
print("샤프 비율:", -result_hist.fun)  # 음수로 설정한 것을 되돌리기 위해 부호를 변경


최적 비중: [0.00000000e+00 1.27899643e-16 3.04212120e-01 7.26504532e-17
 6.95787880e-01]
샤프 비율: 0.3074437371662639


In [91]:
weight_r = pd.DataFrame()

weight_r['historical'] = np.round(result_hist.x, 4)
weight_r['historical'] = weight_r['historical'].apply(float)

weight_r['capm'] = np.round(result_capm.x, 4)
weight_r['capm'] = weight_r['capm'].apply(float)

weight_r

,historical,capm
0,0.0000,0.2788
1,0.0000,0.2915
2,0.3042,0.0379
3,0.0000,0.0796
4,0.6958,0.3122


In [41]:
# implied equilibrium excess returns
lambda_ = 1.5

data = [['INTC', 153.42],
        ['AEP',  19.2],
        ['AMZN', 36.62],
        ['MRK',  125.5],
        ['XOM',  505.49]]

columns = ['stock', 'market_cap']

market_weights = pd.DataFrame(data, columns=columns)
market_weights = market_weights.set_index('stock')
market_weights['market_weights'] = market_weights['market_cap'] / market_weights['market_cap'].sum()

implied_excess_returns_subset = (2 * lambda_ * market_weights['market_weights']).values
implied_excess_returns = np.dot(implied_excess_returns_subset, covariance_matrix_array)
implied_excess_returns

array([0.00922284, 0.0032406 , 0.00738709, 0.0058629 , 0.00635328])

In [55]:
# manager's view

data = [['INTC', 0, 1],
        ['AEP',  1, 0],
        ['AMZN', 0, -1],
        ['MRK',  0, 0],
        ['XOM',  -1, 0]]

columns = ['stock', 0.01, 0.0175]

views = pd.DataFrame(data, columns=columns)
views = views.set_index('stock')
views = views.T

# omerga = uncertainty of view
omerga = 1 * np.dot(np.dot(views.values, covariance_matrix_array), views.T.values)
omerga


array([[ 0.00513993, -0.00123837],
       [-0.00123837,  0.01682516]])